# Completion 생성

## 라이브러리 호출

In [4]:
## 라이브러리 호출
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import random

# Warning 메세지를 뜨지 않게 해줌
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

## 함수생성

### 고객 정보 변환 함수

In [5]:
def calculate_customer_info(info, end_date, customer_grade = '성장형',market_kospi = True):
    
    customer_info_dict = {'투자성향':customer_grade}
    
    # 종목 추출
    name = []
    code = []
    for key in info.keys():
        # 이름과 종목 코드 분리
        name.append(key.split(' (')[0])
        code_raw = key.split(' (')[1][:-1]
        
        # 한국 종목인 경우 ".KS"를 붙임
        if code_raw.isdigit():  # 코드가 숫자로만 구성된 경우
            code.append(f"{code_raw}.KS")
        else:
            code.append(code_raw)
 
    # 이중 딕셔너리에 저장
    customer_data = {
        "종목이름": name,
        "종목코드": code,
        "종목비율": list(info.values())
    }
    
    customer_info_dict.update({'주식종목':{"종목이름": name,"종목비율": list(info.values())}})
    
    
    # 종가정보 추출
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
    start_datetime = (end_datetime - timedelta(days=5*365))
    start_date = start_datetime.strftime('%Y-%m-%d')
    
    data = {}
    for i, symbol in enumerate(customer_data['종목코드']):
        stock_name = customer_data['종목이름'][i]
        try:
            symbol_data = yf.Ticker(symbol).history(start=start_date, end=end_date)
            symbol_data.index = pd.to_datetime(symbol_data.index.date)
            data.update({f'{stock_name}':symbol_data['Close']})
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    stock_df = pd.DataFrame(data)
    
    
    periods = {
    '3개월': [63,13],  # 3개월 (63 거래일, 13주)
    '6개월': [126,26], # 6개월 (126 거래일, 26주)
    '1년': [252,52],   # 1년 (252 거래일, 52주)
    '3년': [756,156]   # 3년 (756 거래일, 156주)
    }
    
    
    # 수익률, 베타 생성
    daily_returns_dict = {}
    avg_returns_dict = {}
    cum_returns_dict = {}
    avg_kospi_period_dict = {}
    avg_sp500_period_dict = {}
    
    betas = {period: {} for period in periods.keys()}
    
    kospi_df = yf.Ticker('^KS11').history(start=start_date, end=end_date)  
    kospi_df.index = pd.to_datetime(kospi_df.index.date)
    
    sp500_df = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
    sp500_df.index = pd.to_datetime(sp500_df.index.date)
    
    daily_kospi_returns = ((kospi_df  / kospi_df.shift(1) -1)*100).fillna(0)
    daily_sp500_returns = ((sp500_df  / sp500_df.shift(1) -1)*100).fillna(0)
    
    for period, days in periods.items():
        days = days[0]
        if len(stock_df) >= days:
            stock_df_period = stock_df[-days:]
            daily_returns  = ((stock_df_period  / stock_df_period.shift(1) -1)*100).fillna(0) #일평균 수익률 계산
            daily_returns_dict[period] = daily_returns
            avg_returns_dict[f'평균수익률({period})'] = np.round(np.dot(customer_data['종목비율'],(daily_returns.mean(axis = 0))*100),2) #평균수익률을 딕셔너리에 저장
            cum_returns_dict[f'누적수익률({period})'] = np.round(np.dot(customer_data['종목비율'],((stock_df_period.iloc[-1] / stock_df_period.iloc[0])-1)*100),2) #수익률을 딕셔너리에 저장
        else:
            avg_returns_dict[period] = None
            cum_returns_dict[period] = None
        
        
        #베타 생성
        kospi_df_period = kospi_df[-days:]
        sp500_df_period = sp500_df[-days:]
        
        avg_kospi_period_dict[f'kospi평균수익률({period})'] = daily_kospi_returns[-days:]['Close'].mean()
        avg_sp500_period_dict[f'sp500평균수익률({period})'] = daily_sp500_returns[-days:]['Close'].mean()
        
        stock_returns = (daily_returns/100).groupby(pd.Grouper(freq='W-FRI')).apply(lambda x: np.log(1 + x / 100).sum())
        kospi_returns = kospi_df_period.pct_change().fillna(0).groupby(pd.Grouper(freq='W-FRI')).apply(lambda x: np.log(1 + x / 100).sum())
        sp500_returns = sp500_df_period.pct_change().fillna(0).groupby(pd.Grouper(freq='W-FRI')).apply(lambda x: np.log(1 + x / 100).sum())
        
        common_index = stock_returns.index.intersection(kospi_returns.index)
        common_index = common_index.intersection(sp500_returns.index)
        
        stock_returns = stock_returns.loc[common_index]
        kospi_returns = kospi_returns.loc[common_index]
        sp500_returns = sp500_returns.loc[common_index]
        
        for i, stock_code in enumerate(customer_data['종목코드']):
            if (stock_code[-2:]=='KS'):
                market_returns = kospi_returns['Close']
            else:
                market_returns = sp500_returns['Close']
            stock_returns_single = stock_returns.iloc[:,i]
            cov_mat = np.cov(stock_returns_single, market_returns)
            covariance = cov_mat[0,1]
            market_variance = cov_mat[1,1]
            beta = covariance / market_variance
            betas[period][stock_code] = beta
            
    def weighted_average(returns, weights, stocks):
        return np.round(sum(returns[stocks[i]] * weights[i] for i in range(len(stocks))),2)
    portfolio_betas = {f'베타({period})': weighted_average(betas[period], customer_data['종목비율'], customer_data['종목코드'])
                            for period in betas}

    customer_info_dict.update(avg_returns_dict)
    # customer_info_dict.update(cum_returns_dict)
    customer_info_dict.update(portfolio_betas)
    
    
    
    # 표준편차 생성
    cov_matrix_dict = {}
    weekly_returns_dict = {}
    portfolio_std_dict = {}
    
    for period, daily_returns in daily_returns_dict.items():
        # 주간 평균 로그수익률 계산
        weekly_returns = daily_returns.groupby(pd.Grouper(freq='W-FRI')).apply(lambda x: np.log(1 + x / 100).sum())
        weekly_returns_dict[period] = weekly_returns
        
        # 공분산 행렬 계산    
        cov_matrix = weekly_returns.cov() 
        cov_matrix_dict[period] = cov_matrix

    # 포트폴리오의 주간 표준편차 계산
    for period, weekly_returns in weekly_returns_dict.items():
        portfolio_std = np.sqrt(np.dot(customer_data['종목비율'], np.dot(cov_matrix_dict[period], customer_data['종목비율']))) * np.sqrt(52)
        portfolio_std_dict[f'표준편차({period})'] = np.round(portfolio_std * 100,2)  # 백분율로 표시하기 위해 100을 곱함
        
    customer_info_dict.update(portfolio_std_dict)
    
    
        
    # 샤프지수, 젠센알파 생성
    tbill = yf.Ticker("^IRX").history(start = start_date, end = end_date)
    risk_free_rates = {'3개월':tbill['Close'].iloc[:90].mean(),
                        '6개월':tbill['Close'].iloc[:180].mean(),
                        '1년':tbill['Close'].iloc[:365].mean(),
                        '3년':tbill['Close'].iloc[:1095].mean()}
    
    portfolio_sharpe_dict = {}
    portfolio_alpha_dict = {}

    for period, risk_free_rate in risk_free_rates.items():
        average_return = avg_returns_dict[f'평균수익률({period})']  # 연환산된 포트폴리오의 평균 수익률
        portfolio_std = portfolio_std_dict[f'표준편차({period})']
        
        # 샤프지수 생성
        sharpe_ratio = (average_return - risk_free_rate) / portfolio_std
        portfolio_sharpe_dict[f'샤프지수({period})'] = np.round(sharpe_ratio, 2)
        
        # 젠센알파 생성
        if market_kospi:
            average_market_return  = avg_kospi_period_dict[f'kospi평균수익률({period})']
        else:
            average_market_return  = avg_sp500_period_dict[f'sp500평균수익률({period})']
        period_beta = portfolio_betas[f'베타({period})']
        alpha = average_return - (risk_free_rate + period_beta * (average_market_return - risk_free_rate))
        portfolio_alpha_dict[f'젠센알파({period})'] = np.round(alpha,2)
        
    customer_info_dict.update(portfolio_sharpe_dict)
    customer_info_dict.update(portfolio_alpha_dict)

        
        
    # 주당 평균가격 생성    
    average_price = stock_df.iloc[-1].copy()
    exchange = yf.download('KRW=X',start=start_date, end=end_date)
    for i, stock_code in enumerate(customer_data['종목코드']):
        if (stock_code[-2:] == "KS"):  # 코드가 KS로 끝날경우
            pass
        else:
            average_price[i] = average_price[i] * exchange['Close'][-1] # 미국주식의 경우 원화로 환산
    average_price = np.dot(average_price, customer_data['종목비율'])

    customer_info_dict.update({'주당평균가격':np.round(average_price,2)})


    # 배당률 생성
    dividend = []
    for stock_code in customer_data['종목코드']:
        dividend.append(yf.Ticker(stock_code).info.get('dividendYield'))
    dividend = pd.Series(dividend).fillna(0)
    average_dividend = np.dot(dividend, customer_data['종목비율'])
    
    customer_info_dict.update({'평균배당률':np.round(average_dividend,4)})
    
    
    # 시리즈 형태로 반환
    customer_info = pd.Series(customer_info_dict)
    
    return customer_info

### 고객정보와 펀드비교 데이터프레임 생성 함수

In [6]:
def fund_rank(customer_info, fund_data, period = '6개월',weight = [1,2,2]):
    recomm_fund = fund_data[['펀드명','위험등급','종목',f'평균수익률({period})',f'베타({period})',
                             f'표준편차({period})',f'샤프지수({period})',f'젠센알파({period})','좌당가격']].copy()
    
    # 0과 공백을 NaN으로 변환, 실수형으로 변경
    recomm_fund.iloc[:,3:-1] = recomm_fund.iloc[:,3:-1].replace('', np.nan).astype(float).replace(0, np.nan)
    
    #겹치는 종목 수
    recomm_fund['종목'] = recomm_fund['종목'].apply(lambda x: len(set(customer_info['주식종목']['종목이름']) & set(x['종목구분'])))  
      
    # 투자성향 -> 성장형, 성장추구형, 위험중립형, 안정추구형, 안정형
    위험등급 = ['매우 높은 위험', '높은 위험', '다소 높은 위험', '낮은 위험']

    if ((customer_info['투자성향'] == '성장형') or (customer_info['투자성향'] == '성장추구형')):
        recomm_fund['위험등급'] = recomm_fund['위험등급'].apply(lambda x: x in 위험등급)
    elif(customer_info['투자성향'] == '위험중립형'):
        recomm_fund['위험등급'] = recomm_fund['위험등급'].apply(lambda x: x in 위험등급[1:])
    elif(customer_info['투자성향'] == '안정추구형'):
        recomm_fund['위험등급'] = recomm_fund['위험등급'].apply(lambda x: x in 위험등급[2:])
    else:
        recomm_fund['위험등급'] = recomm_fund['위험등급'].apply(lambda x: x in 위험등급[3:])
    
    recomm_fund = recomm_fund[recomm_fund['위험등급']]
    
    # 고객의 포트폴리오와 펀드 지표의 차이 계산
    recomm_fund[f'평균수익률({period})'] = recomm_fund[f'평균수익률({period})'].apply(lambda x:  x - customer_info[f'평균수익률({period})'])
    recomm_fund[f'샤프지수({period})'] = recomm_fund[f'샤프지수({period})'].apply(lambda x:  x - customer_info[f'샤프지수({period})'])
    recomm_fund[f'젠센알파({period})'] = recomm_fund[f'젠센알파({period})'].apply(lambda x:  x - customer_info[f'젠센알파({period})'])
    recomm_fund[f'베타({period})'] = recomm_fund[f'베타({period})'].apply(lambda x: customer_info[f'베타({period})'] - x)
    recomm_fund[f'표준편차({period})'] = recomm_fund[f'표준편차({period})'].apply(lambda x: customer_info[f'표준편차({period})'] - x)

    # 전날의 종가를 좌당가격으로 저장
    recomm_fund['좌당가격'] = recomm_fund['좌당가격'].apply(lambda x: x['기준가'][0])

    # 평균을 내기 위해 스케일링
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(recomm_fund.iloc[:, 2:-1])
    scaled_df = pd.concat([recomm_fund.iloc[:,:2],pd.DataFrame(scaled_values,index = recomm_fund.index),recomm_fund.iloc[:,-1]],axis=1)
    scaled_df.columns = recomm_fund.columns
    
    # 평균을 통해 최종스코어 생성
    scaled_df['return_score'] = scaled_df[[f'평균수익률({period})',f'샤프지수({period})',f'젠센알파({period})']].mean(axis=1)
    scaled_df['variation_scroe'] = scaled_df[[f'베타({period})',f'표준편차({period})']].mean(axis=1)
    
    scaled_df[f'점수({period})'] = (weight[0] * scaled_df['종목'] 
                                  + weight[1] * scaled_df['return_score']
                                  + weight[2] * scaled_df['variation_scroe']) / sum(weight)
    
    # 최종스코어를 기준으로 데이터프레임을 정렬, 비교를 위해 고객정보와 합침
    scaled_df = scaled_df.sort_values(by=f'점수({period})',ascending=False)

    recomm_fund = recomm_fund.reindex(scaled_df.index)
    compare_df = pd.concat([customer_info.to_frame().T, fund_data.iloc[scaled_df.index]], ignore_index=True, sort=False)
    # compare_df = compare_df[['펀드명', '위험등급','투자성향','주식종목','종목',
    #                          f'평균수익률({period})',
    #                          f'샤프지수({period})',f'젠센알파({period})', 
    #                          f'표준편차({period})',f'베타({period})']]
    compare_df[[f'평균수익률({period})',
                f'샤프지수({period})',f'젠센알파({period})', 
                f'표준편차({period})',f'베타({period})']] = compare_df[[f'평균수익률({period})',
                                                                    f'샤프지수({period})',f'젠센알파({period})', 
                                                                    f'표준편차({period})',f'베타({period})']].replace('', np.nan).astype(float)   
    return scaled_df, recomm_fund, compare_df

### 고객 포트폴리오 및 펀드정보 텍스트 변환 함수

In [7]:
def info_to_text(customer_info,fund_data,fund_name,period):
    # 해당 펀드명에 해당하는 행 찾기
    펀드_정보 = fund_data[fund_data['펀드명'] == fund_name].iloc[0]
    
    if 펀드_정보.empty:
        return "해당 펀드명을 찾을 수 없습니다."
    
    customer_text = f"""현재 고객님의 포트폴리오 정보입니다. 고객님의 투자성향은 {customer_info['투자성향']}이며,
평균수익률은 {period} 기준 {customer_info[f'평균수익률({period})']}입니다.
베타지수는 {period} 기준 {customer_info[f'베타({period})']}입니다.
표준편차 {period} 기준 {customer_info[f'표준편차({period})']}입니다.
샤프지수는 {period} 기준 {customer_info[f'샤프지수({period})']}입니다.
젠센알파는 {period} 기준 {customer_info[f'젠센알파({period})']}입니다.
고객님께서 투자하신 종목의 주당 평균 가격은 {customer_info['주당평균가격']}이며, 종목들의 평균 배당률은 {customer_info['평균배당률']}입니다.
"""
    # 텍스트 생성
    text = f"""현재 추천받으신 펀드 {펀드_정보['펀드명']}의 정보입니다.{펀드_정보['기준일자']} 기준, 펀드의 좌당금액은 {펀드_정보['좌당가격']['기준가'][0]}원입니다.
추천받으신 펀드의 위험등급은 {펀드_정보['위험등급']} 이며, 펀드 유형은 {펀드_정보['펀드유형']}입니다. 투자지역은 {펀드_정보['투자지역']}로 구성되어 있습니다.
해당 펀드의 평균 수익률은 {period} 기준 {펀드_정보[f'평균수익률({period})']}입니다.
표준편차는 {period} 기준 {펀드_정보[f'표준편차({period})']}입니다.
샤프지수는 {period} 기준 {펀드_정보[f'샤프지수({period})']}입니다.
베타는 {period} 기준 {펀드_정보[f'베타({period})']}입니다.
젠센알파는 {period} 기준 {펀드_정보[f'젠센알파({period})']}입니다.
해당 펀드는 고객님의 투자 성향과 투자종목을 기준으로 한 포트폴리오 기반으로 추천 되었습니다.
"""
    

    return customer_text.strip(), text.strip()

### 답변 생성 함수

In [25]:
# 추세를 기반으로 비교 문구 생성
def generate_trend_comparison(recomm_fund, fund_name, period = '6개월'):
    comparisons = []
    portfolio_info = recomm_fund.iloc[0,:]    # 가지고 있는거
    fund_info = recomm_fund[recomm_fund['펀드명']==fund_name].iloc[0]   # 추천한거

    수익률 = np.round((fund_info[f'평균수익률({period})'] - portfolio_info[f'평균수익률({period})']),2)
    샤프지수 = np.round((fund_info[f'샤프지수({period})'] - portfolio_info[f'샤프지수({period})']),2)
    표준편차 = np.round((fund_info[f'표준편차({period})'] - portfolio_info[f'표준편차({period})']),2)
    베타 = np.round((fund_info[f'베타({period})'] - portfolio_info[f'베타({period})']),2)
    베타_fund = np.round(fund_info[f'베타({period})'],2)
    젠센알파 = np.round((fund_info[f'젠센알파({period})'] - portfolio_info[f'젠센알파({period})']),2)
    
    투자성향 = portfolio_info['투자성향']
    투자지역_fund = fund_info['투자지역']
    
    위험등급 = fund_info['위험등급']

    comparisons.append(f'{period} 기준으로 고객님의 포트폴리오와 비교해보았을 때, 추천해드린 펀드 \'{fund_name}\'의 특징을 설명해드리겠습니다.\n')

    ## 수익률
    # 평균수익률, 샤프지수, 젠센알파 종합 평가
    if 수익률 > 0 and 샤프지수 > 0 and 젠센알파 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n평균수익률이 {abs(수익률)}만큼 더 높고, 샤프지수가 {abs(샤프지수)}만큼 높으며, 젠센알파가 {abs(젠센알파)}만큼 높습니다. \n이 펀드는 투자 위험 대비 더 높은 수익률을 보이고, 시장 대비 실제 수익률도 높다고 해석할 수 있습니다.\n')
    elif 수익률 > 0 and 샤프지수 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n평균수익률이 {abs(수익률)}만큼 더 높고, 샤프지수가 {abs(샤프지수)}만큼 높습니다. \n이 펀드는 투자 위험 대비 더 높은 수익률을 보이는 펀드라고 해석할 수 있습니다.\n')
    elif 수익률 > 0 and 젠센알파 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n평균수익률이 {abs(수익률)}만큼 더 높고, 젠센알파가 {abs(젠센알파)}만큼 높습니다. \n이 펀드는 시장 대비 실제 수익률이 높다고 해석할 수 있습니다.\n')
    elif 샤프지수 > 0 and 젠센알파 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n샤프지수가 {abs(샤프지수)}만큼 높고, 젠센알파가 {abs(젠센알파)}만큼 높습니다. \n이 펀드는 투자 위험 대비 더 높은 수익률을 보이며, 시장 대비 실제 수익률도 높다고 볼 수 있습니다.\n')
    elif 수익률 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n평균수익률이 {abs(수익률)}만큼 더 높습니다.\n')
    elif 샤프지수 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n샤프지수가 {abs(샤프지수)}만큼 높습니다. \n이 펀드는 투자 위험 대비 더 높은 수익률을 보이는 펀드라고 해석할 수 있습니다.\n')
    elif 젠센알파 > 0:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오와 비교했을 때, \n젠센알파가 {abs(젠센알파)}만큼 높습니다. \n이 펀드는 시장 대비 실제 수익률이 높다고 해석할 수 있습니다.\n')
    elif np.isnan(수익률) and np.isnan(샤프지수) and np.isnan(젠센알파):
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) {period}의 수익률, 샤프지수, 젠센알파 값을 제공하지 않고 있어 수익성을 평가할 수 없습니다.\n')
    else:
        comparisons.append(f'\n**수익성** \n저희가 추천해드린 {fund_name}은(는) 고객님의 포트폴리오보다 수익성과, 시장 대비, 투자위험 대비 수익률이 모두 낮습니다.\n')

    ## 변동성
    if 표준편차 > 0 and abs(베타_fund) > 1 and 베타 > 0:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}, 베타가 {abs(베타)}만큼 높습니다. \n이는 펀드의 가격 변동성과 시장대비 변동성이 크고, 시장의 변동성에 더 민감하다는 것을 의미하여 공격적인 투자가 가능한 펀드입니다.\n')
    elif 표준편차 > 0 and abs(베타_fund) > 1:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}높지만, 베타가 {abs(베타)}만큼 낮습니다. \n이는 펀드의 가격 변동성이 크고 시장의 변동성에 더 민감하지만, 시장대비 변동성 작다는 것을 의미합니다.\n')
    elif 표준편차 > 0 and 베타 > 0:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}, 베타가 {abs(베타)}만큼 높습니다. \n이는 펀드의 가격 변동성이 크고 시장의 변동성에 더 민감하지만, 시장대비 변동성이 작다는 것을 의미합니다.\n')
    elif abs(베타_fund) > 1 and 베타 > 0:
        comparisons.append(f'\n**변동성** \n이 펀드는 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}만큼 낮지만, 베타가 {abs(베타)}만큼 높습니다. \n가격의 변동성이 작지만, 시장대비 변동성이 크고, 시장 변동성에 더 민감하다는 것을 의미하여 공격적인 투자가 가능한 펀드입니다.\n')
    elif 표준편차 > 0 and (~np.isnan(베타_fund)):
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}높지만, 베타가 {abs(베타)}만큼 낮습니다. \n이는 펀드의 가격 변동성이 크지만, 시장대비 변동성 작고, 시장의 변동성에 덜 민감하다는 것을 의미하여 비교적 안정적인 투자가 가능한 펀드입니다.\n')
    elif 베타 > 0:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}낮지만, 베타가 {abs(베타)}만큼 높습니다. \n이는 펀드의 시장대비 변동성 크지만, 가격 변동성이 작고, 시장의 변동성에 덜 민감하다는 것을 의미합니다.\n')
    elif abs(베타_fund) > 1:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}만큼 낮고, 베타가 {abs(베타)}만큼 낮습니다. \n이는 펀드의 가격의 변동성과 시장대비 변동성이 작지만, 시장의 변동성에 민감하다는 것을 의미하여 주의가 필요합니다.\n')
    elif np.isnan(표준편차) and np.isnan(베타_fund) and np.isnan(베타):
        comparisons.append(f'\n**변동성** \n이 펀드는 {period}의 표준편차와 베타 값을 제공하지 않고 있어 변동성을 평가할 수 없습니다.\n')
    else:
        comparisons.append(f'\n**변동성** \n이 펀드는 베타가 {베타_fund}이고, 고객님의 포트폴리오보다 표준편차가 {abs(표준편차)}만큼 낮고, 베타가 {abs(베타)}만큼 낮습니다. \n이는 펀드의 가격의 변동성과 시장대비 변동성이 작고, 시장의 변동성에 덜 민감하다는 것을 의미하여 안정적인 투자가 가능합니다.\n')
    
    ## 투자성향
    comparisons.append(f'\n**투자성향** \n저희 {fund_name}은(는) \'{위험등급}\'의 성향을 보이고 있기 때문에, 고객님의 성향인 \'{투자성향}\'에 맞는 투자를 진행할 수 있습니다.\n')

    ## 투자지역
    if 투자지역_fund != '':
        comparisons.append(f'\n**투자지역** \n{fund_name}은(는) {투자지역_fund} 지역에 분할 투자하고 있는 상품입니다.\n')

    ## 수익률과 변동성
    
    변동지수 = 0 
    if 베타 < 0 :
        변동지수 += 1
    if 표준편차 < 0:
        변동지수 += 1
    
    수익지표 = 0
    if 수익률 > 0:
        수익지표 += 1
    if 샤프지수 > 0:
        수익지표 += 1
    if 젠센알파 > 0:
        수익지표 += 1
    
    if (수익지표 >= 2) and (변동지수 < 1):
        comparisons.append(f'\n**종합** \n저희가 제시한 {fund_name}에 투자하신다면, 수익성이 좋고 안정적인 투자를 진행할 수 있습니다.')
    elif (수익지표 >= 2) and (변동지수 >= 1):
        comparisons.append(f'\n**종합** \n저희가 제시한 {fund_name}에 투자하신다면, 위험부담이 있지만 수익성이 좋은 투자를 진행할 수 있습니다.')
    elif (수익지표 < 2) and (변동지수 < 1):
        comparisons.append(f'\n**종합** \n저희가 제시한 {fund_name}에 투자하신다면, 수익성이 높진 않지만, 안정적인 투자를 진행할 수 있습니다.')
    elif (수익지표 < 2) and (변동지수 >= 1):
        comparisons.append(f'\n**종합** \n저희가 제시한 {fund_name}에 투자하신다면, 수익성이 낮지만, 공격적인 투자를 진행할 수 있습니다.')

    return ' '.join(comparisons)

## Completion 생성

### 펀드데이터 불러오기

In [9]:
fund_data = pd.read_json('./data/펀드_데이터_0721.json')

### 고객데이터 생성

### for문을 통해 모든 펀드에 대한 정보 생성

In [10]:
info1 = {
    '에이피알 (214610)': 0.05,
    '삼성전자 (005930)': 0.25,
    'SK하이닉스 (000660)': 0.25,
    'LG화학 (051910)': 0.05,
    '카카오 (035720)': 0.05,
    '네이버 (035420)': 0.20,
    '현대차 (005380)': 0.10,
    '기아 (000270)': 0.05
}

info2 = {
    'Thermo Fisher Scientific Inc (TMO)': 0.20,
    'Apple Inc (AAPL)': 0.25,
    'Microsoft Corp (MSFT)': 0.05,
    'Amazon.com Inc (AMZN)': 0.10,
    'Alphabet Inc (GOOGL)': 0.10,
    'Tesla Inc (TSLA)': 0.15,
    'NVIDIA Corp (NVDA)': 0.10,
    'Facebook Inc (META)': 0.15
}

info3 = {
    '에이피알 (214610)': 0.05,
    '삼성전자 (005930)': 0.25,
    'SK하이닉스 (000660)': 0.15,
    'LG화학 (051910)': 0.05,
    'Thermo Fisher Scientific Inc (TMO)': 0.10,
    'Apple Inc (AAPL)': 0.15,
    'Microsoft Corp (MSFT)': 0.05,
    'Amazon.com Inc (AMZN)': 0.05,
    'Alphabet Inc (GOOGL)': 0.10,
    'Tesla Inc (TSLA)': 0.05
}

info4 = {
    '삼성바이오로직스 (207940)': 0.10,
    '셀트리온 (068270)': 0.10,
    'POSCO홀딩스 (005490)': 0.15,
    'KB금융 (105560)': 0.20,
    '신한지주 (055550)': 0.10,
    '하나금융지주 (086790)': 0.10,
    '우리금융지주 (316140)': 0.10,
    '기업은행 (024110)': 0.15
}

info5 = {
    'LG에너지솔루션 (373220)': 0.20,
    '삼성SDI (006400)': 0.20,
    'SK이노베이션 (096770)': 0.15,
    '에코프로비엠 (247540)': 0.05,
    '엘앤에프 (066970)': 0.10,
    '천보 (278280)': 0.10,
    '포스코케미칼 (003670)': 0.15,
    '동화기업 (025900)': 0.05
}

info6 = {
    'JP Morgan Chase & Co (JPM)': 0.10,
    'Bank of America Corp (BAC)': 0.10,
    'Wells Fargo & Co (WFC)': 0.15,
    'Goldman Sachs Group Inc (GS)': 0.25,
    'Morgan Stanley (MS)': 0.05,
    'Citigroup Inc (C)': 0.15,
    'American Express Co (AXP)': 0.15,
    'U.S. Bancorp (USB)': 0.05
}

info7 = {
    '삼성전자 (005930)': 0.10,
    'SK하이닉스 (000660)': 0.05,
    '네이버 (035420)': 0.10,
    'KB금융 (105560)': 0.05,
    'Apple Inc (AAPL)': 0.10,
    'Microsoft Corp (MSFT)': 0.20,
    'Amazon.com Inc (AMZN)': 0.25,
    'Alphabet Inc (GOOGL)': 0.15
}

In [11]:
infos = [info1, info2, info3, info4, info5, info6, info7]
markets = [True, False, False,True,True,False,False]
periods = ['3개월','6개월','1년']
customer_grades = ['성장형','위험중립형','안정추구형'] #총 7*3*3 = 63건의 정보 생성

# 질문 문구 리스트
questions = [
    '나의 포트폴리오와 너가 추천해준 펀드는 어떤 차이가 있어?',
    '너가 추천해준 펀드랑 내 포트폴리오를 비교해줘',
    '추천펀드랑 포트폴리오 비교해줘',
    '추천해준 이유가 뭐야?',
    '추천해준 펀드가 어떤 펀드인지 자세하게 설명해줘',
    '왜 추천해줬어?',
    '추천펀드의 특징이 뭐야?',
    '추천펀드가 왜 좋아?' 
]

# 빈 데이터프레임 생성
columns = ['c_id', 't_id', 'Text', 'Completion']
final_df = pd.DataFrame(columns=columns)
random.seed(1004)
fund_idx = 0

for customer_grade in customer_grades:
    for period in periods:
        for market, info in zip(markets, infos):
            # 고객 정보 계산
            customer_info = calculate_customer_info(info, end_date='2024-07-21', customer_grade=customer_grade, market_kospi=market)
            scaled_df, recomm_fund, compare_df = fund_rank(customer_info, fund_data, period)
            
            if (customer_grade == '안정추구형'):
                fund_names = compare_df['펀드명'][1:51]
            else: 
                fund_names = compare_df['펀드명'][1:201]

            for fund_name in fund_names:
                # 고객 포트폴리오, 펀드 정보 설명 생성
                customer_text, fund_text = info_to_text(customer_info, fund_data, fund_name, period)

                # 새로운 데이터 추가
                final_df.loc[fund_idx, 'Text'] = f"# 고객 포트폴리오 설명:\n{customer_text}\n# 펀드 정보 설명:\n{fund_text}"

                # 랜덤으로 질문 문구 선택
                question = random.choice(questions)
                
                # 답변 생성
                answer = generate_trend_comparison(compare_df, fund_name, period)
                
                # 데이터프레임 업데이트
                final_df.loc[fund_idx, 'Completion'] = f'\n#질문 : {question} \n#답 : {answer}'
                
                fund_idx += 1
                
# DataFrame에 데이터 추가
final_df['c_id'] = range(len(final_df))
final_df['t_id'] = 0


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [19]:
# final_df.to_csv('./data/completion_v3.csv',index=False,encoding='utf-8')

## 모델 평가용 Completion 생성

In [21]:
info8 = {
    '삼성바이오로직스 (207940)': 0.20,
    '셀트리온 (068270)': 0.15,
    'POSCO홀딩스 (005490)': 0.05,
    'KB금융 (105560)': 0.20,
    'Apple Inc (AAPL)': 0.10,
    'Microsoft Corp (MSFT)': 0.10,
    'Amazon.com Inc (AMZN)': 0.05,
    'Alphabet Inc (GOOGL)': 0.15
}

info9 = {
    'LG에너지솔루션 (373220)': 0.10,
    '삼성SDI (006400)': 0.15,
    'SK이노베이션 (096770)': 0.05,
    '에코프로비엠 (247540)': 0.10,
    'Apple Inc (AAPL)': 0.20,
    'Microsoft Corp (MSFT)': 0.15,
    'Amazon.com Inc (AMZN)': 0.15,
    'Alphabet Inc (GOOGL)': 0.10
}

In [24]:
# 펀드 데이터 불러오기
fund_data = pd.read_json('./data/펀드_데이터_0721.json')

# 고객정보 생성
customer_grade = '성장형'
period = '6개월'

# 고객 정보변환 및 추천펀드 생성
customer_info = calculate_customer_info(info8, end_date='2024-07-21', customer_grade=customer_grade, market_kospi=True)
scaled_df, recomm_fund, compare_df = fund_rank(customer_info, fund_data, period)

# 가장 추천하는 펀드명 추출
fund_name = compare_df.loc[3,'펀드명']

# 고객 포트폴리오, 펀드 정보 설명 생성
customer_text, fund_text = info_to_text(customer_info, fund_data, fund_name, period)

# 답변 생성
answer = generate_trend_comparison(compare_df, fund_name, period)

Text = f"# 고객 포트폴리오 설명:\n{customer_text}\n# 펀드 정보 설명:\n{fund_text}"
print(Text)

Completion = f'#답 : {answer}'
print('\n',Completion)

[*********************100%%**********************]  1 of 1 completed


# 고객 포트폴리오 설명:
현재 고객님의 포트폴리오 정보입니다. 고객님의 투자성향은 성장형이며,
평균수익률은 6개월 기준 11.48입니다.
베타지수는 6개월 기준 1.43입니다.
표준편차 6개월 기준 19.57입니다.
샤프지수는 6개월 기준 0.51입니다.
젠센알파는 6개월 기준 11.98입니다.
고객님께서 투자하신 종목의 주당 평균 가격은 371891.75이며, 종목들의 평균 배당률은 0.0107입니다.
# 펀드 정보 설명:
현재 추천받으신 펀드 미래에셋미국배당프리미엄증권모투자신탁(주식)의 정보입니다.[기준일 : 2024.07.19] 기준, 펀드의 좌당금액은 2,266.22원입니다.
추천받으신 펀드의 위험등급은 다소 높은 위험 이며, 펀드 유형은 (해외주식형)입니다. 투자지역은 로 구성되어 있습니다.
해당 펀드의 평균 수익률은 6개월 기준 21.29입니다.
표준편차는 6개월 기준 9.72입니다.
샤프지수는 6개월 기준 4.02입니다.
베타는 6개월 기준 0.36입니다.
젠센알파는 6개월 기준 25.18입니다.
해당 펀드는 고객님의 투자 성향과 투자종목을 기준으로 한 포트폴리오 기반으로 추천 되었습니다.

 #답 : 6개월 기준으로 고객님의 포트폴리오와 비교해보았을 때, 추천해드린 펀드 '미래에셋미국배당프리미엄증권모투자신탁(주식)'의 특징을 설명해드리겠습니다.
 
**수익성** 
저희가 추천해드린 미래에셋미국배당프리미엄증권모투자신탁(주식)은(는) 고객님의 포트폴리오와 비교했을 때, 
평균수익률이 9.81만큼 더 높고, 샤프지수가 3.51만큼 높으며, 젠센알파가 13.2만큼 높습니다. 
이 펀드는 투자 위험 대비 더 높은 수익률을 보이고, 시장 대비 실제 수익률도 높다고 해석할 수 있습니다.
 
**변동성** 
이 펀드는 베타가 0.36이고, 고객님의 포트폴리오보다 표준편차가 9.85만큼 낮고, 베타가 1.07만큼 낮습니다. 
이는 펀드의 가격의 변동성과 시장대비 변동성이 작지만, 시장의 변동성에 덜 민감하다는 것을 의미하여 안정적인 투자가 가능합니다.
 
**